# Testing a Flask server for ML predictions

This notebook walks through developing a flask server for serving the predictions from `Batch_Data_Training.ipynb`.


In [165]:
# Load libraries
import numpy as np
import xgboost as xgb
from flask import Flask, abort, jsonify, request
import pandas as pd
import json

## Load the trained model

In [5]:
bst = xgb.Booster() #init model
bst.load_model("../0002.model") # load data

# Test Model predictions

Before deploying the server, I'm testing how to format the data for predictions using a sample dataset.

In [166]:
df = pd.read_csv("../datasample/base1run1.csv")
df.head()

,Time,Fermentation volume,Dissolved oxygen concentration,Dissolved CO2 concentration,Reactor temperature,pH,Feed rate,Feed temperature,Agitator power,Cooling/heating medium flow rate,...,Dissolved oxygen concentration prior2,Dissolved CO2 concentration prior2,Reactor temperature prior2,pH prior2,Feed rate prior2,Feed temperature prior2,Agitator power prior2,Cooling/heating medium flow rate prior2,Cumulative base flow prior2,Cumulative acid flow prior2
0,5.760000,95.278900,1.153800,0.500000,297.910000,5.002000,0.000000,296.040000,29.560000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
1,50.894837,94.390448,1.105594,1.599498,297.970325,5.082031,0.060215,296.008069,29.972236,21.337598,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
2,110.841631,96.565049,1.094002,2.427862,297.904680,4.994837,0.059749,295.931418,29.985320,46.104148,...,1.153800,0.500000,297.910000,5.002000,-3.197442e-14,296.040000,29.560000,5.440093e-15,0.000024,0.000005
3,157.120855,98.218117,1.086594,2.750265,298.005726,5.005043,0.060091,295.950427,30.117436,69.003419,...,1.105594,1.599498,297.970325,5.082031,6.021547e-02,296.008069,29.972236,2.133760e+01,0.002733,0.000007
4,165.481139,98.515685,1.088136,2.724702,298.031139,5.001626,0.059852,295.971708,29.820820,72.172255,...,1.094002,2.427862,297.904680,4.994837,5.974876e-02,295.931418,29.985320,4.610415e+01,0.004266,0.000007


The data need to be formatted as a DMatrix in order to predict. I then run the predictions and convert the results to a JSON object for return.

In [167]:
m1=df.as_matrix()[0:3,1:]
if len(m1.shape) == 1:
    inputdata=xgb.DMatrix(m1[np.newaxis,:],feature_names=bst.feature_names)
else:
    inputdata=xgb.DMatrix(m1,feature_names=bst.feature_names)

preds = bst.predict(inputdata)
#print(preds)
l1=preds[:,0].tolist()
l2=df.as_matrix()[0:3,0]
pd.DataFrame({'Time':l2,'predictions':l1}).to_json(orient='records')

'[{"Time":5.76,"predictions":0.9074475765},{"Time":50.8948374253,"predictions":0.0001362579},{"Time":110.8416310874,"predictions":0.0004667733}]'

# Flask Server

I now make a server that will respond to post requests, make a prediction on the incomming data, and return the results as a JSON object.

In [164]:
app = Flask(__name__)

@app.route('/api',methods=['POST'])
def make_predict():
    dataobject = request.get_json(force=True)
    #print(dataobject)
    predict_request = list(dataobject['data'].values())
    predict_request = np.array(predict_request[1:-1], dtype='f')
    if len(predict_request.shape) == 1:
        
        times = np.array(predict_request[0], dtype='f')
        inputdata=xgb.DMatrix(predict_request[np.newaxis,:],feature_names=bst.feature_names)
    else:
        times = np.array(predict_request[:,0], dtype='f')
        inputdata=xgb.DMatrix(predict_request,feature_names=bst.feature_names)

    preds = bst.predict(inputdata)
    #print(preds)
    output = preds[:,0].tolist()
    
    outputdf = pd.DataFrame({'Time':times,'predictions':output}).to_json(orient='records')
    
    returnmsg = jsonify(deviceId = dataobject['deviceId'],
                        datetime = dataobject['datetime'],
                        protocol = dataobject['protocol'],
                        batchID = dataobject['data']['batchID'],
                        results=json.loads(outputdf))
    
    return returnmsg

@app.route("/")
def hello():
    return "Python Flask POST endpoint."

#
# Running app in Jupyter notebook:
#

from werkzeug.serving import run_simple
run_simple('localhost', 9000, app)


127.0.0.1 - - [12/Jun/2017 13:22:04] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2017 13:22:04] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2017 13:22:04] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2017 13:22:04] "POST /api HTTP/1.1" 200 -
